# Task 5: House Prices Dataset — Data Cleaning (Pandas)

**Goal:** Clean the raw dataset using Pandas (missing values, duplicates, datatypes, feature creation) and export a cleaned CSV.

**Deliverables:**
- Task5_Cleaning.ipynb
- cleaned_data.csv
- 5–10 markdown notes inside the notebook


## 1) Environment Setup
Google Colab is used to ensure an easy, reproducible environment with common data science libraries pre-installed.

In [ ]:
import pandas as pd
import numpy as np

# If you're using Colab, upload your CSV to the session (Files pane) or mount Google Drive.
# Dataset used in this task:
file_name = 'Bangalore.csv'

df = pd.read_csv(file_name)
print('Shape:', df.shape)
df.head()

## 2) Dataset Overview
We use `.head()` and `.info()` to understand the structure, column types, and whether there are missing values.

In [ ]:
df.info()

## 3) Missing Values Check
We identify missing values per column using `isnull().sum()` so we can decide how to clean them.

In [ ]:
missing = df.isnull().sum().sort_values(ascending=False)
print(missing.head(20))
print('\nTotal missing values:', int(missing.sum()))

## 4) Remove Duplicates
Duplicate rows can distort results. We drop duplicates and verify the number of rows removed.

In [ ]:
before_rows = df.shape[0]
df = df.drop_duplicates()
after_rows = df.shape[0]

print('Rows before:', before_rows)
print('Rows after :', after_rows)
print('Removed    :', before_rows - after_rows)

## 5) Clean Missing Values
Strategy:
- Numeric columns → fill using **median** (robust to outliers)
- Categorical columns → fill using **mode**
- If a column has very high missingness, it may be dropped (not needed here if missing is low).

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(exclude=[np.number]).columns

# Fill numeric with median
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# Fill categorical with mode
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])

print('Remaining missing values:', int(df.isnull().sum().sum()))

## 6) Datatype Conversion
We convert date-like columns to datetime if present. This enables correct time-based calculations.

In [ ]:
# Convert any column containing 'date' in its name to datetime
for col in df.columns:
    if 'date' in col.lower():
        df[col] = pd.to_datetime(df[col], errors='coerce')

# If conversion creates missing dates (NaT), fill with mode (optional)
for col in df.columns:
    if pd.api.types.is_datetime64_any_dtype(df[col]) and df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])

print('Datetime columns:', [c for c in df.columns if pd.api.types.is_datetime64_any_dtype(df[c])])
df.info()

## 7) Feature Engineering (New Column)
We create a new column to demonstrate transformation skills. If a price column exists, we create `Price_Band` (Low/Medium/High) using quantiles.

In [ ]:
price_cols = [c for c in df.columns if 'price' in c.lower()]
print('Possible price columns:', price_cols)

if price_cols:
    price_col = price_cols[0]
    # Create 3 bands using quantiles
    df['Price_Band'] = pd.qcut(df[price_col], q=3, labels=['Low', 'Medium', 'High'])
    print(df[['Price_Band', price_col]].head())
else:
    print('No price column detected. If your dataset has price, rename it to include "price" or set price_col manually.')

## 8) Final Quality Checks
We confirm final shape, missing values, and basic descriptive statistics.

In [ ]:
print('Final shape:', df.shape)
print('Total missing values:', int(df.isnull().sum().sum()))
df.describe(include='all').T.head(15)

## 9) Export Cleaned Dataset
We save the cleaned dataset as `cleaned_data.csv` and confirm the file is created.

In [ ]:
output_file = 'cleaned_data.csv'
df.to_csv(output_file, index=False)
print('Saved:', output_file)

## Outcome
This notebook demonstrates how Pandas can replace manual Excel cleaning workflows for large datasets, improving speed and reproducibility.